<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 220px; height: 150px; vertical-align: middle;">
            <img src="./assets/aaa.png" width="220" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Autonomous Traders</h2>
            <span style="color:#ff7800;">An equity trading simulation to illustrate autonomous agents powered by tools and resources from MCP servers.
            </span>
        </td>
    </tr>
</table>

An equity trading simulation, with 4 Traders and a Researcher, powered by a 6 MCP servers with 44 tools & 2 resources:

1. Our home-made Accounts MCP server (written by our engineering team!)
2. Fetch (get webpage via a local headless browser)
3. Memory
4. Brave Search
5. Financial data from Polygon
6. The return of our Push Notification tool ❤️

And a resource to read information about the trader's account, and their investment strategy.

This lab looks at a new python module, `traders.py` that will manage a single trader on our trading floor.

We will experiment and explore in the lab, and then migrate to a python module when we're ready.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="./assets/stop.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">One more time --</h2>
            <span style="color:#ff7800;">Please do not use this for actual trading decisions!!
            </span>
        </td>
    </tr>
</table>

In [ ]:
pip install dotenv mcp openai-agents polygon-api-client

In [1]:
import os
from dotenv import load_dotenv
from agents import Agent, Runner, trace, Tool
from agents.mcp import MCPServerStdio
from IPython.display import Markdown, display
from datetime import datetime
from accounts_client import read_accounts_resource, read_strategy_resource
from accounts import Account

load_dotenv(override=True)

True

[non-fatal] Tracing: request failed: timed out
[non-fatal] Tracing: request failed: timed out
[non-fatal] Tracing: request failed: timed out
[non-fatal] Tracing: max retries reached, giving up on this batch.
[non-fatal] Tracing: request failed: [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: Hostname mismatch, certificate is not valid for 'api.openai.com'. (_ssl.c:1010)
[non-fatal] Tracing: request failed: [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: Hostname mismatch, certificate is not valid for 'api.openai.com'. (_ssl.c:1010)
[non-fatal] Tracing: request failed: [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: Hostname mismatch, certificate is not valid for 'api.openai.com'. (_ssl.c:1010)
[non-fatal] Tracing: max retries reached, giving up on this batch.


### Let's start by gathering the MCP servers for our trader and researcher

In [2]:
from market import is_paid_polygon, is_realtime_polygon

load_dotenv(override=True)

brave_env = {"BRAVE_API_KEY": os.getenv("BRAVE_API_KEY")}
polygon_api_key = os.getenv("POLYGON_API_KEY")

# The MCP server for the Trader to read Market Data

if is_paid_polygon or is_realtime_polygon:
    market_mcp = {"command": "uvx","args": ["--from", "git+https://github.com/polygon-io/mcp_polygon@master", "mcp_polygon"], "env": {"POLYGON_API_KEY": polygon_api_key}}
else:
    market_mcp = ({"command": "uv", "args": ["run", "market_server.py"]})


# The full set of MCP servers for the trader: Accounts, Push Notification and the Market

trader_mcp_server_params = [
    {"command": "uv", "args": ["run", "accounts_server.py"]},
    {"command": "uv", "args": ["run", "push_server.py"]},
    market_mcp
]

def researcher_mcp_server_params(name: str):
    return [
        {"command": "uvx", "args": ["mcp-server-fetch"]},
        {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-brave-search"], "env": {"BRAVE_API_KEY": os.getenv("BRAVE_API_KEY")}},
        {"command": "npx", "args": ["-y", "mcp-memory-libsql"], "env": {"LIBSQL_URL": f"file:{name}.db"}}
    ]
trader_mcp_servers = [MCPServerStdio(params) for params in trader_mcp_server_params]
researcher_mcp_servers = [MCPServerStdio(params) for params in researcher_mcp_server_params("amar")]
mcp_servers = trader_mcp_servers + researcher_mcp_servers

### Now let's make a Researcher Agent to do market research

And turn it into a tool

In [3]:
async def get_researcher(mcp_servers) -> Agent:
    instructions = f"""You are a financial researcher. You are able to search the web for interesting financial news,
look for possible trading opportunities, and help with research.
Based on the request, you carry out necessary research and respond with your findings.
Take time to make multiple searches to get a comprehensive overview, and then summarize your findings.
If there isn't a specific request, then just respond with investment opportunities based on searching latest news.
The current datetime is {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
"""
    researcher = Agent(
        name="Researcher",
        instructions=instructions,
        model="gpt-4o-mini",
        mcp_servers=mcp_servers,
    )
    return researcher


async def get_researcher_tool(mcp_servers) -> Tool:
    researcher = await get_researcher(mcp_servers)
    return researcher.as_tool(
            tool_name="Researcher",
            tool_description="This tool researches online for news and opportunities, \
                either based on your specific request to look into a certain stock, \
                or generally for notable financial news and opportunities. \
                Describe what kind of research you're looking for."
        )

In [ ]:
research_question = "What's the latest news on MathWorks?"

for server in researcher_mcp_servers:
    await server.connect()
researcher = await get_researcher(researcher_mcp_servers)
with trace("Researcher"):
    print("Researcher is ready.", researcher)
    result = await Runner.run(researcher, research_question)
display(Markdown(result.final_output))


### Look at the trace

https://platform.openai.com/traces

In [4]:
from mcp import StdioServerParameters
from database import init_db
init_db()
amar_initial_strategy = "You are a day trader that aggressively buys and sells shares based on news and market conditions."
Account.get("Amar").reset(amar_initial_strategy)
params = StdioServerParameters(command="uv", args=["run", "accounts_server.py"], env=None)
account_server = MCPServerStdio({"command": "uv", "args": ["run", "accounts_server.py"]})

async def read_accounts_resource(name, server) -> str:
    """Read account information for the given name.
    
    Args:
        name: The name of the account holder
        
    Returns:
        Account report as a string
    """
    print(f"Reading account resource for {name}")
    
    result = await account_server.session.read_resource(f"accounts://accounts_server/{name}")
    print("Result: ", result)
    return result.contents[0].text

await account_server.connect()
display(Markdown(await read_accounts_resource(name="Amar", server=account_server)))

Reading account resource for Amar
Result:  meta=None contents=[TextResourceContents(uri=AnyUrl('accounts://accounts_server/Amar'), mimeType='text/plain', text='{"name": "amar", "balance": 10000.0, "strategy": "You are a day trader that aggressively buys and sells shares based on news and market conditions.", "holdings": {}, "transactions": [], "portfolio_value_time_series": [["2025-05-20 18:26:48", 10000.0]], "total_portfolio_value": 10000.0, "total_profit_loss": 0.0}')]


{"name": "amar", "balance": 10000.0, "strategy": "You are a day trader that aggressively buys and sells shares based on news and market conditions.", "holdings": {}, "transactions": [], "portfolio_value_time_series": [["2025-05-20 18:26:48", 10000.0]], "total_portfolio_value": 10000.0, "total_profit_loss": 0.0}

In [5]:

display(Markdown(await read_accounts_resource("Amar", account_server)))
display(Markdown(await read_strategy_resource("Amar")))

Reading account resource for Amar
Result:  meta=None contents=[TextResourceContents(uri=AnyUrl('accounts://accounts_server/Amar'), mimeType='text/plain', text='{"name": "amar", "balance": 10000.0, "strategy": "You are a day trader that aggressively buys and sells shares based on news and market conditions.", "holdings": {}, "transactions": [], "portfolio_value_time_series": [["2025-05-20 18:26:48", 10000.0], ["2025-05-20 18:26:50", 10000.0]], "total_portfolio_value": 10000.0, "total_profit_loss": 0.0}')]


{"name": "amar", "balance": 10000.0, "strategy": "You are a day trader that aggressively buys and sells shares based on news and market conditions.", "holdings": {}, "transactions": [], "portfolio_value_time_series": [["2025-05-20 18:26:48", 10000.0], ["2025-05-20 18:26:50", 10000.0]], "total_portfolio_value": 10000.0, "total_profit_loss": 0.0}

You are a day trader that aggressively buys and sells shares based on news and market conditions.

### And now - to create our Trader Agent

In [6]:
agent_name = "Amar"

# Using MCP Servers to read resources
account_details = await read_accounts_resource(agent_name, account_server)
strategy = await read_strategy_resource(agent_name)

instructions = f"""
You are a trader that manages a portfolio of shares. Your name is {agent_name} and your account is under your name, {agent_name}.
You have access to tools that allow you to search the internet for company news, check stock prices, and buy and sell shares.
Your investment strategy for your portfolio is:
{strategy}
Your current holdings and balance is:
{account_details}
You have the tools to perform a websearch for relevant news and information.
You have tools to check stock prices.
You have tools to buy and sell shares.
You have tools to save memory of companies, research and thinking so far.
Please make use of these tools to manage your portfolio. Carry out trades as you see fit; do not wait for instructions or ask for confirmation.
"""

prompt = """
Use your tools to make decisions about your portfolio.
Investigate the news and the market, make your decision, make the trades, and respond with a summary of your actions.
"""

Reading account resource for Amar
Result:  meta=None contents=[TextResourceContents(uri=AnyUrl('accounts://accounts_server/Amar'), mimeType='text/plain', text='{"name": "amar", "balance": 10000.0, "strategy": "You are a day trader that aggressively buys and sells shares based on news and market conditions.", "holdings": {}, "transactions": [], "portfolio_value_time_series": [["2025-05-20 18:26:48", 10000.0], ["2025-05-20 18:26:50", 10000.0], ["2025-05-20 18:26:52", 10000.0]], "total_portfolio_value": 10000.0, "total_profit_loss": 0.0}')]


In [7]:
display(Markdown(instructions))


You are a trader that manages a portfolio of shares. Your name is Amar and your account is under your name, Amar.
You have access to tools that allow you to search the internet for company news, check stock prices, and buy and sell shares.
Your investment strategy for your portfolio is:
You are a day trader that aggressively buys and sells shares based on news and market conditions.
Your current holdings and balance is:
{"name": "amar", "balance": 10000.0, "strategy": "You are a day trader that aggressively buys and sells shares based on news and market conditions.", "holdings": {}, "transactions": [], "portfolio_value_time_series": [["2025-05-20 18:26:48", 10000.0], ["2025-05-20 18:26:50", 10000.0], ["2025-05-20 18:26:52", 10000.0]], "total_portfolio_value": 10000.0, "total_profit_loss": 0.0}
You have the tools to perform a websearch for relevant news and information.
You have tools to check stock prices.
You have tools to buy and sell shares.
You have tools to save memory of companies, research and thinking so far.
Please make use of these tools to manage your portfolio. Carry out trades as you see fit; do not wait for instructions or ask for confirmation.


### And to run our Trader

In [10]:
for server in mcp_servers:
    try:
        await server.connect()
    except Exception as e:
        print(f"Failed to connect: {server.params.command} | Error: {e}")

print("All servers connected.")
print("Trader is ready.", [trader_mcp_servers.params.args for trader_mcp_servers in trader_mcp_servers])
print("Researcher is ready.", [researcher_mcp_servers.params.args for researcher_mcp_servers in researcher_mcp_servers])

researcher_tool = await get_researcher_tool(researcher_mcp_servers)

print("Researcher tool is ready.", researcher_tool)
trader = Agent(
    name=agent_name,
    instructions=instructions,
    tools=[researcher_tool],
    mcp_servers=trader_mcp_servers,
    model="gpt-4o-mini",
)
with trace(agent_name):
    print("Trader is ready.", trader)
    result = await Runner.run(trader, prompt)
display(Markdown(result.final_output))

All servers connected.
Trader is ready. [['run', 'accounts_server.py'], ['run', 'push_server.py'], ['--from', 'git+https://github.com/polygon-io/mcp_polygon@master', 'mcp_polygon']]
Researcher is ready. [['mcp-server-fetch'], ['-y', '@modelcontextprotocol/server-brave-search'], ['-y', 'mcp-memory-libsql']]
Researcher tool is ready. FunctionTool(name='Researcher', description="This tool researches online for news and opportunities,                 either based on your specific request to look into a certain stock,                 or generally for notable financial news and opportunities.                 Describe what kind of research you're looking for.", params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'Researcher_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x115501d00>, strict_json_schema=True)
Trader is ready.

### Summary of Actions Made

#### Research Findings
1. **Market Surge**: Stocks surged following a U.S.-China trade agreement, creating promising conditions for tech stocks.
2. **Overall Outlook**: Analysts suggest a cautious optimism in tech, indicating potential growth due to strong demand and economic recovery.

#### Trades Executed
1. **Buy Stocks**:
   - **Apple (AAPL)**: Bought 10 shares at $78.16 due to strong performance expectations.
   - **Amazon (AMZN)**: Bought 5 shares at $53.11, taking advantage of bullish market sentiments.
   - **Alphabet (GOOGL)**: Bought 5 shares at $77.15, expected to benefit from increased ad revenue.
   - **Microsoft (MSFT)**: Bought 5 shares at $26.05, strong demand for cloud services.
   - **Netflix (NFLX)**: Bought 5 shares at $12.02, anticipating post-pandemic subscriber growth.

#### Portfolio Update
- **New Holdings**:
  - AAPL: 10 shares
  - AMZN: 5 shares
  - GOOGL: 5 shares
  - MSFT: 5 shares
  - NFLX: 5 shares

- **Cash Balance After Trades**: $8,376.76
- **Total Portfolio Value**: $9,376.76
- **Total Loss since inception**: -$623.24

#### Summary
Your portfolio has been diversified into key tech stocks following positive market news. The balance is now $8,376.76 with investments positioned for potential gains as the market reacts to trade developments.

### Now it's time to review the Python module made from this:

`mcp_servers.py` is where the MCP servers are specified. You'll notice I've brought in some familiar friends: memory and push notifications!

`templates.py` is where the instructions and messages are set up (i.e. the System prompts and User prompts)

`traders.py` brings it all together.
